Demo of PySpark in VSCode reading from Azure Storage Blobs

In [1]:
from msal import PublicClientApplication
from pyspark.sql import SparkSession

Log-in

In [2]:
app_id = '04b07795-8ddb-461a-bbee-02f9e1bf7b46' # Azure CLI app ID
auth = PublicClientApplication(app_id)
refresh_token = auth.acquire_token_interactive(scopes=['https://storage.azure.com/.default'])['refresh_token']

Start Spark

In [4]:
spark = (
    SparkSession
    .builder
    .master('local[*]')
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-azure:3.4.2,com.azure:azure-storage-blob:12.32.0,org.apache.spark:spark-avro_2.13:4.1.0')
    .config('fs.azure.account.auth.type', 'OAuth')
    .config('fs.azure.account.oauth.provider.type', 'org.apache.hadoop.fs.azurebfs.oauth2.RefreshTokenBasedTokenProvider')
    .config('fs.azure.account.oauth2.refresh.token', refresh_token)
    .config('fs.azure.account.oauth2.client.id', app_id)
    .getOrCreate()
)

print(f'Spark version: {spark.version}')
print(f'Spark UI available at: http://localhost:4040')

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/vscode/.ivy2.5.2/cache
The jars for the packages stored in: /home/vscode/.ivy2.5.2/jars
org.apache.hadoop#hadoop-azure added as a dependency
com.azure#azure-storage-blob added as a dependency
org.apache.spark#spark-avro_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b35b394f-4f99-47d5-a51c-3f546f4cfb7c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-azure;3.4.2 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.3.0 in central
	found commons-codec#commons-codec;1.15 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.4.0 in central
	fo

Spark version: 4.1.0
Spark UI available at: http://localhost:4040


In [ ]:
df = spark.read.format('avro').load('abfss://<container>@<account>.dfs.core.windows.net/path/to/blob.avro')
df.count()

16159